# Telogical Chatbot Framework Guide

This notebook provides a comprehensive guide to understanding the Telogical Chatbot framework, its architecture, and how to integrate your own AI agents into it.

## Table of Contents
1. [Project Overview](#project-overview)
2. [Architecture](#architecture)
3. [AI Agent Integration](#ai-agent-integration)
4. [Service Layer (FastAPI)](#service-layer)
5. [Client Integration](#client-integration)
6. [Front-end Options](#frontend-options)
7. [Docker Deployment](#docker-deployment)
8. [RAG Integration](#rag-integration)
9. [Custom Tool Visualization](#custom-tool-visualization)
10. [End-to-End Examples](#end-to-end-examples)

## 1. Project Overview <a name="project-overview"></a>

The Telogical Chatbot framework is a comprehensive toolkit for building, deploying, and managing AI agents. It provides:

- A modular architecture for integrating various types of AI agents
- A service layer built with FastAPI for exposing agents via API
- Client libraries for interacting with the service
- A Streamlit-based UI for demonstration and interaction
- Docker containerization for easy deployment
- Support for various memory backends (MongoDB, PostgreSQL, SQLite)
- Integration with LangGraph for complex agent workflows

The framework is designed to be flexible, allowing you to use components individually or together as a complete solution.

## 2. Architecture <a name="architecture"></a>

The project follows a modular architecture with several key components:

### Core Components:

```
┌───────────────┐      ┌────────────────┐      ┌────────────────┐
│   AI Agents   │◄─────┤  Service Layer │◄─────┤  Client Layer  │
│  (LLM-based)  │      │   (FastAPI)    │      │                │
└───────┬───────┘      └────────────────┘      └────────────────┘
        │                                             ▲
        │                                             │
        ▼                                             │
┌───────────────┐                           ┌─────────────────┐
│    Memory     │                           │   Front-end     │
│   Backends    │                           │  (Streamlit)    │
└───────────────┘                           └─────────────────┘
```

### Key Directories and Files:

- **src/agents/**: Contains all AI agent implementations
- **src/service/**: FastAPI service layer
- **src/client/**: Client library for interacting with the service
- **src/core/**: Core settings and LLM interfaces
- **src/memory/**: Memory backends for conversation history
- **src/schema/**: Data models and schemas
- **src/streamlit_app.py**: Streamlit front-end
- **docker/**: Docker configuration files

### Key Flows:

1. **Front-end → Client → Service → Agent**: User input flows from the UI through the client to the service to the agent
2. **Agent → Memory**: Conversations are stored in the selected memory backend
3. **Agent → Tools**: Agents can use various tools to perform tasks
4. **Service → Client → Front-end**: Agent responses flow back to the user

## 3. AI Agent Integration <a name="ai-agent-integration"></a>

The framework supports multiple types of agents, which are implemented in the `src/agents/` directory. Let's explore how to integrate your own agent:

### Agent Structure

Agents in this framework are built on the concept of a base agent class that handles common functionality, with specific agent types implementing their unique behavior. The main agent types include:

- **Chatbot** (`src/agents/chatbot.py`): Basic conversational agent
- **RAG Assistant** (`src/agents/rag_assistant.py`): Retrieval-augmented generation agent
- **Research Assistant** (`src/agents/research_assistant.py`): Web search capable agent
- **LangGraph Supervisor** (`src/agents/langgraph_supervisor_agent.py`): Complex workflow agent

### Integrating Your Own Agent

To integrate your own agent, follow these steps:

In [ ]:
# Example: Creating a custom agent
from src.agents.agents import BaseAgent
from src.schema.models import ChatMessage, MessageRole
from typing import List, Optional, Dict, Any

class MyCustomAgent(BaseAgent):
    """Custom agent implementation"""
    
    def __init__(self, model_name: str, **kwargs):
        super().__init__(model_name=model_name, **kwargs)
        # Initialize your custom components here
        self.my_custom_component = None
    
    async def process_message(
        self, 
        messages: List[ChatMessage], 
        stream: bool = False,
        **kwargs
    ) -> ChatMessage:
        """Process a message from the user"""
        # Your custom logic here
        # Example: Prepare messages for the LLM
        prepared_messages = self._prepare_messages(messages)
        
        # Call the LLM
        if stream:
            return await self._stream_llm_response(prepared_messages, **kwargs)
        else:
            return await self._get_llm_response(prepared_messages, **kwargs)
    
    def _prepare_messages(self, messages: List[ChatMessage]) -> List[Dict[str, Any]]:
        """Prepare messages for the LLM"""
        # Your custom message preparation logic
        return [{
            "role": msg.role.value,
            "content": msg.content
        } for msg in messages]

### Registering Your Agent

After creating your agent class, you need to register it with the framework so it can be instantiated through the service layer:

In [ ]:
# Register your agent in src/agents/agents.py
from src.agents.agents import AGENT_REGISTRY

# Add your agent to the registry
AGENT_REGISTRY["my_custom_agent"] = MyCustomAgent

### Adding Tool Support

If your agent needs to use tools, you can add tool support like this:

In [ ]:
from src.agents.tools import BaseTool

# Create a custom tool
class MyCustomTool(BaseTool):
    name = "my_custom_tool"
    description = "A tool that does something useful"
    
    def _run(self, input_text: str) -> str:
        """Execute the tool"""
        # Your tool logic here
        return f"Processed: {input_text}"

# Then in your agent class:
def __init__(self, model_name: str, **kwargs):
    super().__init__(model_name=model_name, **kwargs)
    # Add your custom tool to the agent
    self.tools = [MyCustomTool()]
    
    # Configure your agent to use tools
    self.tool_config = {
        "tools": [tool.to_dict() for tool in self.tools],
        "tool_choice": "auto"
    }

### Visualizing Tool Execution

To visualize tool execution without exposing the actual tools to the user, you can leverage the framework's message tracing capabilities:

In [ ]:
# Example: Adding tool execution tracing to your agent
from src.schema.models import ChatMessage, MessageRole, MessageMetadata

async def process_message(self, messages: List[ChatMessage], **kwargs):
    # ... your existing code ...
    
    # When a tool is executed, add trace information
    tool_name = "my_tool"
    tool_input = "some input"
    tool_output = "some output"
    
    # Create a metadata object with trace information
    metadata = MessageMetadata(
        trace={
            "tool_calls": [{
                "name": tool_name,
                "input": tool_input,
                "output": tool_output
            }]
        }
    )
    
    # Add the metadata to the response message
    response = ChatMessage(
        role=MessageRole.ASSISTANT,
        content="I've analyzed the data.",
        metadata=metadata
    )
    
    return response

## 4. Service Layer (FastAPI) <a name="service-layer"></a>

The service layer is implemented using FastAPI and provides REST and WebSocket endpoints for interacting with agents. Let's explore how to understand and extend the service layer:

### Service Architecture

The service is defined in `src/service/service.py` and provides these key endpoints:

- **POST /chat**: Send a message to an agent and get a response
- **WebSocket /ws/chat**: Stream messages to and from an agent
- **GET /agents**: List available agents
- **GET /health**: Health check endpoint

### Running the Service

The service can be started using the `src/run_service.py` script:

In [ ]:
# Start the service
!python src/run_service.py

### Extending the Service

To add new endpoints or functionality to the service, you can extend the `service.py` file:

In [ ]:
# Example: Adding a new endpoint to the service
from fastapi import APIRouter, Depends, HTTPException
from src.schema.models import ChatRequest, ChatResponse
from src.service.utils import get_agent_instance

# Create a new router (in service.py)
custom_router = APIRouter(prefix="/custom", tags=["custom"])

@custom_router.post("/analyze")
async def analyze_data(request: ChatRequest):
    """Custom endpoint for data analysis"""
    try:
        # Get the agent instance
        agent = get_agent_instance(request.agent_id, request.agent_type)
        
        # Process the request
        messages = request.messages
        # Add a system message indicating this is an analysis request
        messages.insert(0, ChatMessage(
            role=MessageRole.SYSTEM,
            content="This is a data analysis request. Focus on extracting insights."
        ))
        
        # Get the response
        response = await agent.process_message(messages)
        
        return ChatResponse(message=response)
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

# Add the router to the main app
app.include_router(custom_router)

## 5. Client Integration <a name="client-integration"></a>

The client library (`src/client/client.py`) provides a convenient way to interact with the service layer. Here's how to use it:

In [ ]:
# Example: Using the client library
from src.client.client import TelogicalClient
from src.schema.models import ChatMessage, MessageRole

# Create a client instance
client = TelogicalClient(base_url="http://localhost:8000")

# Send a message to an agent
async def chat_with_agent():
    message = ChatMessage(
        role=MessageRole.USER,
        content="What is machine learning?"
    )
    
    response = await client.chat(
        agent_type="chatbot",
        messages=[message]
    )
    
    print(response.content)

# Stream messages from an agent
async def stream_from_agent():
    message = ChatMessage(
        role=MessageRole.USER,
        content="Explain quantum computing in simple terms."
    )
    
    async for chunk in client.chat_stream(
        agent_type="chatbot",
        messages=[message]
    ):
        print(chunk.content, end="")

### Extending the Client

You can extend the client to add support for your custom endpoints:

In [ ]:
# Example: Extending the client for custom endpoints
class ExtendedClient(TelogicalClient):
    """Extended client with custom functionality"""
    
    async def analyze_data(self, messages):
        """Call the custom analyze endpoint"""
        url = f"{self.base_url}/custom/analyze"
        payload = {
            "agent_type": "my_custom_agent",
            "messages": [msg.dict() for msg in messages]
        }
        
        async with self.session.post(url, json=payload) as response:
            response.raise_for_status()
            data = await response.json()
            return ChatMessage(**data["message"])

## 6. Front-end Options <a name="frontend-options"></a>

The framework includes a Streamlit front-end (`src/streamlit_app.py`), but you can integrate with other front-end technologies like React or Vue.

### Using the Streamlit Front-end

The Streamlit app provides a simple interface for interacting with agents:

In [ ]:
# Run the Streamlit app
!streamlit run src/streamlit_app.py

### Integrating with React

To integrate with a React front-end, you would create a React application that communicates with the FastAPI service. Here's an example of how to create a simple React client:

In [ ]:
%%writefile react-client-example.jsx
// Example React component for chatting with the agent
import React, { useState, useEffect, useRef } from 'react';
import axios from 'axios';

const ChatComponent = () => {
  const [messages, setMessages] = useState([]);
  const [input, setInput] = useState('');
  const [loading, setLoading] = useState(false);
  const [traceInfo, setTraceInfo] = useState(null);
  
  const sendMessage = async () => {
    if (!input.trim()) return;
    
    // Add user message to chat
    const userMessage = { role: 'user', content: input };
    setMessages([...messages, userMessage]);
    setInput('');
    setLoading(true);
    
    try {
      // Send message to API
      const response = await axios.post('http://localhost:8000/chat', {
        agent_type: 'chatbot',
        messages: [...messages, userMessage].map(msg => ({
          role: msg.role,
          content: msg.content
        }))
      });
      
      // Add assistant response to chat
      const assistantMessage = response.data.message;
      setMessages([...messages, userMessage, assistantMessage]);
      
      // Check for trace information
      if (assistantMessage.metadata?.trace) {
        setTraceInfo(assistantMessage.metadata.trace);
      }
    } catch (error) {
      console.error('Error sending message:', error);
    } finally {
      setLoading(false);
    }
  };
  
  return (
    <div className="chat-container">
      <div className="messages-container">
        {messages.map((msg, index) => (
          <div key={index} className={`message ${msg.role}`}>
            <div className="message-content">{msg.content}</div>
          </div>
        ))}
        {loading && <div className="loading">AI is thinking...</div>}
      </div>
      
      {traceInfo && (
        <div className="trace-info">
          <h3>Agent Actions</h3>
          {traceInfo.tool_calls?.map((tool, index) => (
            <div key={index} className="tool-call">
              <div className="tool-name">Using tool: {tool.name}</div>
              <div className="tool-input">Input: {tool.input}</div>
              <div className="tool-output">Result: {tool.output}</div>
            </div>
          ))}
        </div>
      )}
      
      <div className="input-container">
        <input
          type="text"
          value={input}
          onChange={(e) => setInput(e.target.value)}
          onKeyPress={(e) => e.key === 'Enter' && sendMessage()}
          placeholder="Type your message..."
          disabled={loading}
        />
        <button onClick={sendMessage} disabled={loading}>
          Send
        </button>
      </div>
    </div>
  );
};

export default ChatComponent;

### Integrating with Node.js

For a Node.js backend that communicates with the FastAPI service, you could create an Express server that acts as a proxy:

In [ ]:
%%writefile nodejs-proxy-example.js
// Example Node.js Express server as a proxy to the FastAPI service
const express = require('express');
const axios = require('axios');
const cors = require('cors');
const app = express();
const port = 3000;

// Enable CORS and JSON parsing
app.use(cors());
app.use(express.json());

// Base URL for the FastAPI service
const API_BASE_URL = 'http://localhost:8000';

// Proxy endpoint for chat
app.post('/api/chat', async (req, res) => {
  try {
    const response = await axios.post(`${API_BASE_URL}/chat`, req.body);
    res.json(response.data);
  } catch (error) {
    console.error('Error proxying request:', error);
    res.status(error.response?.status || 500).json({
      error: error.response?.data || 'Internal server error'
    });
  }
});

// Proxy endpoint for listing agents
app.get('/api/agents', async (req, res) => {
  try {
    const response = await axios.get(`${API_BASE_URL}/agents`);
    res.json(response.data);
  } catch (error) {
    console.error('Error proxying request:', error);
    res.status(error.response?.status || 500).json({
      error: error.response?.data || 'Internal server error'
    });
  }
});

// Serve static files (for the React front-end)
app.use(express.static('public'));

// Start the server
app.listen(port, () => {
  console.log(`Node.js proxy server listening on port ${port}`);
});

## 7. Docker Deployment <a name="docker-deployment"></a>

The framework includes Docker configuration for easy deployment. Let's explore how to use it:

### Docker Compose Setup

The project includes a `compose.yaml` file that defines the services needed to run the application:

In [ ]:
# Start the application using Docker Compose
!docker-compose up -d

### Building Custom Docker Images

If you need to customize the Docker images, you can modify the Dockerfiles in the `docker/` directory and build your own images:

In [ ]:
# Build a custom Docker image for the service
!docker build -f docker/Dockerfile.service -t my-custom-telogical-service .

### Deploying to a Cloud Provider

You can deploy the Docker containers to various cloud providers like AWS, GCP, or Azure. Here's an example for AWS ECS:

In [ ]:
%%writefile deploy-to-ecs.sh
#!/bin/bash

# Build and push Docker images to ECR
aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin $AWS_ACCOUNT_ID.dkr.ecr.us-west-2.amazonaws.com

# Build and push service image
docker build -f docker/Dockerfile.service -t telogical-service .
docker tag telogical-service:latest $AWS_ACCOUNT_ID.dkr.ecr.us-west-2.amazonaws.com/telogical-service:latest
docker push $AWS_ACCOUNT_ID.dkr.ecr.us-west-2.amazonaws.com/telogical-service:latest

# Update ECS service
aws ecs update-service --cluster telogical-cluster --service telogical-service --force-new-deployment

## 8. RAG Integration <a name="rag-integration"></a>

The framework includes support for Retrieval-Augmented Generation (RAG) via the `rag_assistant.py` agent. Let's explore how to use and customize it:

### Setting Up a Vector Database

The framework includes a script for creating a Chroma vector database from documents:

In [ ]:
# Run the script to create a Chroma database
!python scripts/create_chroma_db.py --input_dir /path/to/documents --output_dir data/chroma_db

### Customizing the RAG Assistant

You can customize the RAG assistant by extending the `RAGAssistant` class:

In [ ]:
# Example: Customizing the RAG Assistant
from src.agents.rag_assistant import RAGAssistant
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

class CustomRAGAssistant(RAGAssistant):
    """Custom RAG assistant with specialized retrieval"""
    
    def __init__(self, model_name: str, **kwargs):
        super().__init__(model_name=model_name, **kwargs)
        
        # Initialize custom embeddings
        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
        
        # Initialize vector store with custom settings
        self.vectorstore = Chroma(
            collection_name="custom_docs",
            embedding_function=embeddings,
            persist_directory="data/custom_chroma_db"
        )
        
        # Customize retrieval parameters
        self.top_k = 5
        self.similarity_threshold = 0.7
    
    async def _retrieve_relevant_documents(self, query: str):
        """Custom document retrieval logic"""
        # Add your custom retrieval logic here
        docs = self.vectorstore.similarity_search_with_score(
            query, k=self.top_k
        )
        
        # Filter by similarity threshold
        filtered_docs = [doc for doc, score in docs if score >= self.similarity_threshold]
        
        return filtered_docs

## 9. Custom Tool Visualization <a name="custom-tool-visualization"></a>

To visualize the agent's reasoning process and tool usage without exposing the actual tools to the user, you can implement a custom visualization in your front-end:

### Adding Trace Information to Messages

When your agent uses tools, it can add trace information to the response messages:

In [ ]:
# Example: Adding trace information to messages
from src.schema.models import ChatMessage, MessageRole, MessageMetadata

def create_response_with_trace(content: str, tool_calls: list):
    """Create a response message with trace information"""
    metadata = MessageMetadata(
        trace={
            "tool_calls": tool_calls,
            "reasoning": "I need to search for information about quantum computing."
        }
    )
    
    return ChatMessage(
        role=MessageRole.ASSISTANT,
        content=content,
        metadata=metadata
    )

### Displaying Trace Information in the UI

You can then display this trace information in your UI, whether it's Streamlit, React, or another framework:

In [ ]:
%%writefile streamlit-trace-visualization-example.py
import streamlit as st
from src.client.client import TelogicalClient
from src.schema.models import ChatMessage, MessageRole

# Initialize client
client = TelogicalClient(base_url="http://localhost:8000")

# Initialize session state
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages
for message in st.session_state.messages:
    with st.chat_message(message.role.value):
        st.write(message.content)
        
        # Display trace information if available
        if message.metadata and message.metadata.trace:
            with st.expander("Agent Reasoning"):
                st.write(message.metadata.trace.get("reasoning", ""))
            
            if "tool_calls" in message.metadata.trace:
                for i, tool_call in enumerate(message.metadata.trace["tool_calls"]):
                    with st.expander(f"Tool: {tool_call['name']}"):
                        st.write(f"**Input:** {tool_call['input']}")
                        st.write(f"**Output:** {tool_call['output']}")

# Chat input
if prompt := st.chat_input():
    # Add user message to chat
    user_message = ChatMessage(role=MessageRole.USER, content=prompt)
    st.session_state.messages.append(user_message)
    
    with st.chat_message("user"):
        st.write(prompt)
    
    # Get response from agent
    with st.spinner("Thinking..."):
        response = await client.chat(
            agent_type="research_assistant",  # This agent uses tools
            messages=st.session_state.messages
        )
        
        st.session_state.messages.append(response)
    
    # Display assistant response
    with st.chat_message("assistant"):
        st.write(response.content)
        
        # Display trace information
        if response.metadata and response.metadata.trace:
            with st.expander("Agent Reasoning"):
                st.write(response.metadata.trace.get("reasoning", ""))
            
            if "tool_calls" in response.metadata.trace:
                for i, tool_call in enumerate(response.metadata.trace["tool_calls"]):
                    with st.expander(f"Tool: {tool_call['name']}"):
                        st.write(f"**Input:** {tool_call['input']}")
                        st.write(f"**Output:** {tool_call['output']}")

## 10. End-to-End Examples <a name="end-to-end-examples"></a>

Let's put everything together with an end-to-end example of integrating a custom agent, connecting it to the service, and building a front-end to interact with it:

### Step 1: Create a Custom Agent

In [ ]:
%%writefile src/agents/my_reasoning_agent.py
from typing import List, Dict, Any, Optional
from src.agents.agents import BaseAgent
from src.schema.models import ChatMessage, MessageRole, MessageMetadata
from src.agents.tools import BaseTool

class CalculatorTool(BaseTool):
    """A simple calculator tool"""
    name = "calculator"
    description = "Evaluate mathematical expressions"
    
    def _run(self, input_text: str) -> str:
        """Evaluate a mathematical expression"""
        try:
            # Security: Use safe evaluation to prevent code execution
            result = eval(input_text, {"__builtins__": {}}, {})
            return str(result)
        except Exception as e:
            return f"Error: {str(e)}"

class ReasoningAgent(BaseAgent):
    """Custom agent that shows its reasoning"""
    
    def __init__(self, model_name: str, **kwargs):
        super().__init__(model_name=model_name, **kwargs)
        
        # Initialize tools
        self.tools = [CalculatorTool()]
        
        # Configure the agent to use tools
        self.tool_config = {
            "tools": [tool.to_dict() for tool in self.tools],
            "tool_choice": "auto"
        }
    
    async def process_message(
        self, 
        messages: List[ChatMessage], 
        stream: bool = False,
        **kwargs
    ) -> ChatMessage:
        """Process a message with visible reasoning"""
        # Prepare system message to guide the agent's behavior
        system_message = {
            "role": "system",
            "content": (
                "You are a reasoning agent that explains your thought process. "
                "First, analyze the problem and explain your reasoning step by step. "
                "If calculations are needed, use the calculator tool. "
                "After using tools, provide a final answer."
            )
        }
        
        # Prepare messages for the LLM
        prepared_messages = [system_message] + [{
            "role": msg.role.value,
            "content": msg.content
        } for msg in messages]
        
        # Call the LLM with tool support
        llm_response = await self.llm.chat_with_tools(
            messages=prepared_messages,
            tools=self.tool_config["tools"],
            tool_choice=self.tool_config["tool_choice"],
            stream=False
        )
        
        # Extract the response content and tool calls
        content = llm_response["content"] if llm_response["content"] else ""
        tool_calls = []
        
        # Process tool calls if any
        if "tool_calls" in llm_response:
            for tool_call in llm_response["tool_calls"]:
                tool_name = tool_call["function"]["name"]
                tool_input = tool_call["function"]["arguments"]
                
                # Find the tool
                tool = next((t for t in self.tools if t.name == tool_name), None)
                
                if tool:
                    # Execute the tool
                    tool_output = tool.run(tool_input)
                    
                    # Add to tool calls for tracing
                    tool_calls.append({
                        "name": tool_name,
                        "input": tool_input,
                        "output": tool_output
                    })
        
        # Create a metadata object with trace information
        metadata = MessageMetadata(
            trace={
                "tool_calls": tool_calls,
                "reasoning": "Here's my step-by-step reasoning: ..."  # You could extract this from the content
            }
        )
        
        # Return the response message
        return ChatMessage(
            role=MessageRole.ASSISTANT,
            content=content,
            metadata=metadata
        )

### Step 2: Register the Agent

In [ ]:
%%writefile src/agents/agents.py.updated
# Add this to the end of the existing agents.py file
from src.agents.my_reasoning_agent import ReasoningAgent

# Register the new agent
AGENT_REGISTRY["reasoning_agent"] = ReasoningAgent

### Step 3: Create a Front-end to Visualize the Agent's Reasoning

In [ ]:
%%writefile src/reasoning_app.py
import streamlit as st
import asyncio
from src.client.client import TelogicalClient
from src.schema.models import ChatMessage, MessageRole

st.set_page_config(page_title="Reasoning Agent", page_icon="🧠")

# Initialize client
client = TelogicalClient(base_url="http://localhost:8000")

st.title("Reasoning Agent Demo")
st.markdown(
    "This demo shows how an AI agent can reveal its reasoning process and tool usage "
    "without exposing the actual tools to the user."
)

# Initialize session state
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages
for message in st.session_state.messages:
    with st.chat_message(message.role.value):
        st.markdown(message.content)
        
        # Display reasoning and tool calls if available
        if message.role.value == "assistant" and message.metadata and message.metadata.trace:
            if "reasoning" in message.metadata.trace:
                with st.expander("View Reasoning"):
                    st.markdown(message.metadata.trace["reasoning"])
            
            if "tool_calls" in message.metadata.trace and message.metadata.trace["tool_calls"]:
                steps = message.metadata.trace["tool_calls"]
                with st.expander(f"View Steps ({len(steps)} steps)"):
                    for i, step in enumerate(steps):
                        st.markdown(f"**Step {i+1}: Using {step['name']}**")
                        st.markdown(f"Input: `{step['input']}`")
                        st.markdown(f"Result: `{step['output']}`")
                        st.divider()

# Chat input
if prompt := st.chat_input("Ask a question..."):
    # Add user message to chat
    user_message = ChatMessage(role=MessageRole.USER, content=prompt)
    st.session_state.messages.append(user_message)
    
    with st.chat_message("user"):
        st.markdown(prompt)
    
    # Get response from agent
    with st.spinner("Thinking..."):
        # Using asyncio.run is not ideal in Streamlit, but this is just for demonstration
        async def get_response():
            return await client.chat(
                agent_type="reasoning_agent",
                messages=st.session_state.messages
            )
        
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)
        response = loop.run_until_complete(get_response())
        st.session_state.messages.append(response)
    
    # Display assistant response
    with st.chat_message("assistant"):
        st.markdown(response.content)
        
        # Display reasoning and tool calls
        if response.metadata and response.metadata.trace:
            if "reasoning" in response.metadata.trace:
                with st.expander("View Reasoning"):
                    st.markdown(response.metadata.trace["reasoning"])
            
            if "tool_calls" in response.metadata.trace and response.metadata.trace["tool_calls"]:
                steps = response.metadata.trace["tool_calls"]
                with st.expander(f"View Steps ({len(steps)} steps)"):
                    for i, step in enumerate(steps):
                        st.markdown(f"**Step {i+1}: Using {step['name']}**")
                        st.markdown(f"Input: `{step['input']}`")
                        st.markdown(f"Result: `{step['output']}`")
                        st.divider()

# Add a sidebar with instructions
with st.sidebar:
    st.header("Instructions")
    st.markdown(
        "Ask questions that might require calculations or step-by-step reasoning. "
        "The agent will show its work."
    )
    st.markdown("Example questions:")
    st.markdown("- What is 127 * 345?")
    st.markdown("- If I have 125 apples and give 37 away, how many do I have left?")
    st.markdown("- What's the compound interest on $1000 at 5% for 3 years?")

### Step 4: Run Everything

To run the complete system, you would:

1. Start the FastAPI service: `python src/run_service.py`
2. Run the Streamlit app: `streamlit run src/reasoning_app.py`

Alternatively, you can use Docker Compose to start everything:

```bash
docker-compose up -d
```

Then access the application at http://localhost:8501 for the Streamlit UI or http://localhost:8000/docs for the FastAPI documentation.

## Summary

The Telogical Chatbot framework provides a comprehensive solution for building, deploying, and managing AI agents. Key takeaways:

1. **Modular Architecture**: Components can be used individually or together
2. **Flexible Agent Integration**: Easy to add your own custom agents
3. **Multiple Front-end Options**: Use Streamlit or integrate with React/Node.js
4. **Tool Visualization**: Show agent reasoning without exposing tools
5. **Docker Deployment**: Easy containerization and deployment
6. **Memory Options**: Multiple backends for conversation history
7. **LangGraph Support**: Complex multi-agent workflows

To integrate your own agent, focus on:
1. Creating an agent class that extends BaseAgent
2. Registering it in the AGENT_REGISTRY
3. Customizing the front-end to visualize your agent's behavior

For deployment, use the provided Docker configuration and adjust as needed for your specific requirements.